# Setup from Sherlock inputs

This example shows how to create an Icepak project from Sherlock
files (STEP and CSV) and an AEDB board.

Keywords: **Icepak**, **Sherlock**.

## Perform imports and define constants

Perform required imports.

In [1]:
import os
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_sherlock

Define constants

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Set paths and define input files and variables

Set paths.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
input_dir = download_sherlock(local_path=temp_folder.name)

Define input files.

In [4]:
material_name = "MaterialExport.csv"
component_properties = "TutorialBoardPartsList.csv"
component_step = "TutorialBoard.stp"
aedt_odb_project = "SherlockTutorial.aedt"

Define variables that are needed later.

In [5]:
aedt_odb_design_name = "PCB"
outline_polygon_name = "poly_14188"

Define input files with paths.

In [6]:
material_list = os.path.join(input_dir, material_name)
component_list = os.path.join(input_dir, component_properties)
validation = os.path.join(temp_folder.name, "validation.log")
file_path = os.path.join(input_dir, component_step)
project_name = os.path.join(temp_folder.name, component_step[:-3] + "aedt")

## Create Icepak model

In [7]:
ipk = ansys.aedt.core.Icepak(
    project=project_name, version=AEDT_VERSION, non_graphical=NG_MODE
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.22.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_afee94ea-7ac2-4e66-ab77-85f84fe8af40.log is enabled.


...

PyAEDT INFO: Ansoft.ElectronicsDesktop.2025.2 version started with process ID 4996.


PyAEDT INFO: Project TutorialBoard has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Icepak_G9P' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


Disable autosave to speed up the import.

In [8]:
ipk.autosave_disable()

True

Import a PCB from an AEDB file.

In [9]:
odb_path = os.path.join(input_dir, aedt_odb_project)
ipk.create_pcb_from_3dlayout(
    component_name="Board", project_name=odb_path, design_name=aedt_odb_design_name
)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Closing the AEDT Project SherlockTutorial


PyAEDT INFO: Project SherlockTutorial closed correctly


PyAEDT INFO: PCB component correctly created in Icepak.


Create an offset coordinate system to match ODB++ with the Sherlock STEP file.
The thickness is computed from the ``"Board"`` component. (``"Board1"`` is the
instance name of the ``"Board"`` native component and is used to offset the coordinate system.)

In [10]:
bb = ipk.modeler.user_defined_components["Board1"].bounding_box
stackup_thickness = bb[-1] - bb[2]
ipk.modeler.create_coordinate_system(
    origin=[0, 0, stackup_thickness / 2], mode="view", view="XY"
)

Import the board components from an MCAD file and remove the PCB object as it is already
imported with the ECAD.

In [11]:
ipk.modeler.import_3d_cad(file_path, refresh_all_ids=False)
ipk.modeler.delete_objects_containing("pcb", False)

PyAEDT INFO: Step file C:\Users\ansys\AppData\Local\Temp\tmp_78g03xk.ansys\sherlock\TutorialBoard.stp imported


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Deleted 1 Objects: PCB.


PyAEDT INFO: Deleted 1 objects


True

Modify the air region. Padding values are passed in this order: [+X, -X, +Y, -Y, +Z, -Z]

In [12]:
ipk.mesh.global_mesh_region.global_region.padding_values = [20, 20, 20, 20, 300, 300]

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp_78g03xk.ansys\TutorialBoard.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp_78g03xk.ansys\TutorialBoard.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.012603282928466797


PyAEDT WARNING: No mesh operation found.


PyAEDT WARNING: No mesh region found.


...

PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


PyAEDT WARNING: Property Command is read-only.


## Assign materials and power dissipation conditions from Sherlock

Use the Sherlock file to assign materials.

In [13]:
ipk.assignmaterial_from_sherlock_files(
    component_file=component_list, material_file=material_list
)

PyAEDT INFO: Adding new material to the Project Library: bariumtitanate


PyAEDT INFO: Material has been added in Desktop.


PyAEDT INFO: Adding new material to the Project Library: epoxyencapsulant


PyAEDT INFO: Material has been added in Desktop.


PyAEDT INFO: Adding new material to the Project Library: laminate-bga


...

PyAEDT INFO: Dataset Mass_Density_L1ZZRT doesn't exist.


PyAEDT INFO: Dataset $Mass_Density_L1ZZRT created successfully.


PyAEDT INFO: Dataset Youngs_Modulus_T89BKI doesn't exist.


PyAEDT INFO: Dataset $Youngs_Modulus_T89BKI created successfully.


True

Delete objects with no material assignments.

In [14]:
no_material_objs = ipk.modeler.get_objects_by_material(material="")
ipk.modeler.delete(assignment=no_material_objs)
ipk.save_project()

PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Deleted 158 Objects: COMP_R58,COMP_R57,COMP_R56,COMP_R55,COMP_R54,COMP_R53,COMP_R52,COMP_R51,COMP_R50,COMP_R49,COMP_R48,COMP_R47,COMP_R46,COMP_R45,COMP_R44,COMP_R43,COMP_R42,COMP_R41,COMP_R40,COMP_R39,COMP_R38,COMP_R37,COMP_R36,COMP_R35,COMP_R34,COMP...


PyAEDT INFO: Project TutorialBoard Saved correctly


True

Assign power blocks from the Sherlock file.

In [15]:
total_power = ipk.assign_block_from_sherlock_file(csv_name=component_list)

PyAEDT INFO: Boundary Block COMP_C1 has been created.


PyAEDT INFO: Boundary Block COMP_C2 has been created.


PyAEDT INFO: Boundary Block COMP_C3 has been created.


PyAEDT INFO: Boundary Block COMP_C4 has been created.


PyAEDT INFO: Boundary Block COMP_C5 has been created.


...

PyAEDT INFO: Boundary Block COMP_U13 has been created.


PyAEDT INFO: Boundary Block COMP_U14 has been created.


PyAEDT INFO: Boundary Block COMP_U15 has been created.


PyAEDT INFO: Boundary Block COMP_U16 has been created.


PyAEDT INFO: Blocks inserted with total power 2.2W.


In [16]:
# ## Assign openings
#
# Assign opening boundary condition to all the faces of the region.
ipk.assign_openings(ipk.modeler.get_object_faces("Region"))

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp_78g03xk.ansys\TutorialBoard.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp_78g03xk.ansys\TutorialBoard.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.07868289947509766


PyAEDT INFO: Boundary Opening Opening_ZGP1HC has been created.


## Create simulation setup
### Set global mesh settings

In [17]:
ipk.globalMeshSettings(
    3,
    gap_min_elements="1",
    noOgrids=True,
    MLM_en=True,
    MLM_Type="2D",
    edge_min_elements="2",
    object="Region",
)

True

### Add postprocessing object
Create the point monitor.

In [18]:
point1 = ipk.monitor.assign_point_monitor(
    point_position=ipk.modeler["COMP_U10"].top_face_z.center, monitor_name="Point1"
)

Create a line for reporting after the simulation.

In [19]:
line = ipk.modeler.create_polyline(
    points=[
        ipk.modeler["COMP_U10"].top_face_z.vertices[0].position,
        ipk.modeler["COMP_U10"].top_face_z.vertices[2].position,
    ],
    non_model=True,
)

### Solve
To solve quickly, the maximum iterations are set to 20. For better accuracy, you
can increase the maximum to at least 100.

In [20]:
setup1 = ipk.create_setup()
setup1.props["Solution Initialization - Y Velocity"] = "1m_per_sec"
setup1.props["Radiation Model"] = "Discrete Ordinates Model"
setup1.props["Include Gravity"] = True
setup1.props["Secondary Gradient"] = True
setup1.props["Convergence Criteria - Max Iterations"] = 100

Check for intersections using validation and fix them by
assigning priorities.

In [21]:
ipk.assign_priority_on_intersections()

PyAEDT INFO: Project TutorialBoard Saved correctly


PyAEDT INFO: 0 Intersections have been found. Applying Priorities


True

## Release AEDT

In [22]:
ipk.save_project()
ipk.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project TutorialBoard Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [23]:
temp_folder.cleanup()